In [4]:
from pyannote.database import registry
from pyannote.audio.tasks import SpeakerDiarization
from google.cloud import secretmanager
from pyannote.audio import Model
from pyannote.audio import Pipeline

def access_secret_version(project_id, secret_id, version_id):
    """
    Access the payload for the given secret version if one exists. The version
    can be a version number as a string (e.g. "5") or an alias (e.g. "latest").
    """

    client = secretmanager.SecretManagerServiceClient()

    # Build the resource name of the secret version.
    name = f"projects/{project_id}/secrets/{secret_id}/versions/{version_id}"

    # Access the secret version.
    response = client.access_secret_version(name=name)

    # Return the decoded payload.
    return response.payload.data.decode('UTF-8')

HF_KEY = access_secret_version(1076126751560, "huggingface-api-key", "latest")

In [16]:
registry.load_database("/Users/StephenWitkowski/Projects/broadcastify-transcription/data/broadcastify_database.yaml")
protocol = registry.get_protocol('MyDatabase.SpeakerDiarization')

ValueError: not enough values to unpack (expected 3, got 2)

In [9]:

pretrained = pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization@2.1", use_auth_token=HF_KEY)

torchvision is not available - cannot save figures
Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/c4c8ceafcbb3a7a280c2d357aee9fbc9b0be7f9b/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1. Bad things might happen unless you revert torch to 1.x.


In [21]:
registry.get_database("MyDatabase")

In [24]:
database = registry.get_database('MyDatabase')

In [42]:
protocol = registry.get_protocol('MyDatabase.SpeakerDiarization.MyProtocol')

In [69]:
import os
os.environ["PYANNOTE_DATABASE_CONFIG"] = "/Users/StephenWitkowski/Projects/broadcastify-transcription/data/broadcastify_database.yaml"
from pyannote.database import get_protocol, FileFinder
dataset = get_protocol("MyDatabase.SpeakerDiarization.MyProtocol", {"audio": FileFinder()})
     

In [76]:
from pyannote.audio import Model
model = Model.from_pretrained("pyannote/segmentation", use_auth_token=True)

Lightning automatically upgraded your loaded checkpoint from v1.5.4 to v2.0.5. To apply the upgrade to your files permanently, run `python -m pytorch_lightning.utilities.upgrade_checkpoint --file ../../../../.cache/torch/pyannote/models--pyannote--segmentation/snapshots/7d5cf7bca4dcac7f943eb834bec0906a90da8c97/pytorch_model.bin`


Model was trained with pyannote.audio 0.0.1, yours is 2.1.1. Bad things might happen unless you revert pyannote.audio to 0.x.
Model was trained with torch 1.10.0+cu102, yours is 2.0.1. Bad things might happen unless you revert torch to 1.x.


In [89]:
Segmentation?

Init signature:
Segmentation(
    protocol: pyannote.database.protocol.speaker_diarization.SpeakerDiarizationProtocol,
    duration: float = 2.0,
    max_speakers_per_chunk: int = None,
    max_speakers_per_frame: int = None,
    weigh_by_cardinality: bool = False,
    warm_up: Union[float, Tuple[float, float]] = 0.0,
    balance: str = None,
    weight: str = None,
    batch_size: int = 32,
    num_workers: int = None,
    pin_memory: bool = False,
    augmentation: torch_audiomentations.core.transforms_interface.BaseWaveformTransform = None,
    vad_loss: Literal['bce', 'mse'] = None,
    metric: Union[torchmetrics.metric.Metric, Sequence[torchmetrics.metric.Metric], Dict[str, torchmetrics.metric.Metric]] = None,
    max_num_speakers: int = None,
    loss: Literal['bce', 'mse'] = None,
)
Docstring:     
Speaker diarization

Parameters
----------
protocol : SpeakerDiarizationProtocol
    pyannote.database protocol
duration : float, optional
    Chunks duration. Defaults to 2s.
max_spe

In [91]:
from pyannote.audio.tasks import Segmentation
task = Segmentation(
    dataset, 
    duration=2, 
    max_num_speakers=10, 
    batch_size=32,
    num_workers=2, 
    loss="bce", 
    vad_loss="bce")
model.task = task
model.setup(stage="fit")

In [93]:
?Trainer

Init signature:
Trainer(
    *,
    accelerator: Union[str, pytorch_lightning.accelerators.accelerator.Accelerator] = 'auto',
    strategy: Union[str, pytorch_lightning.strategies.strategy.Strategy] = 'auto',
    devices: Union[List[int], str, int] = 'auto',
    num_nodes: int = 1,
    precision: Union[Literal[64, 32, 16], Literal['16-mixed', 'bf16-mixed', '32-true', '64-true'], Literal['64', '32', '16', 'bf16']] = '32-true',
    logger: Union[pytorch_lightning.loggers.logger.Logger, Iterable[pytorch_lightning.loggers.logger.Logger], bool, NoneType] = None,
    callbacks: Union[List[pytorch_lightning.callbacks.callback.Callback], pytorch_lightning.callbacks.callback.Callback, NoneType] = None,
    fast_dev_run: Union[int, bool] = False,
    max_epochs: Optional[int] = None,
    min_epochs: Optional[int] = None,
    max_steps: int = -1,
    min_steps: Optional[int] = None,
    max_time: Union[str, datetime.timedelta, Dict[str, int], NoneType] = None,
    limit_train_batches: Union[int, 

In [92]:
# this takes approximately 15min to run on Google Colab GPU
from types import MethodType
from torch.optim import Adam
from pytorch_lightning.callbacks import (
    EarlyStopping,
    ModelCheckpoint,
    RichProgressBar,
)

# we use Adam optimizer with 1e-4 learning rate
def configure_optimizers(self):
    return Adam(self.parameters(), lr=1e-4)

model.configure_optimizers = MethodType(configure_optimizers, model)

# we monitor diarization error rate on the validation set
# and use to keep the best checkpoint and stop early
monitor, direction = task.val_monitor
checkpoint = ModelCheckpoint(
    monitor=monitor,
    mode=direction,
    save_top_k=1,
    every_n_epochs=1,
    save_last=False,
    save_weights_only=False,
    filename="{epoch}",
    verbose=False,
)
early_stopping = EarlyStopping(
    monitor=monitor,
    mode=direction,
    min_delta=0.0,
    patience=10,
    strict=True,
    verbose=False,
)

callbacks = [RichProgressBar(), checkpoint, early_stopping]

# we train for at most 20 epochs (might be shorter in case of early stopping)
from pytorch_lightning import Trainer
trainer = Trainer(accelerator="cpu", 
                  callbacks=callbacks, 
                  max_epochs=20,
                  gradient_clip_val=0.5)
trainer.fit(model)
     

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


┏━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃   ┃ Name              ┃ Type             ┃ Params ┃      In sizes ┃                                   Out sizes ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ 0 │ sincnet           │ SincNet          │ 42.6 K │ [1, 1, 32000] │                                [1, 60, 115] │
│ 1 │ lstm              │ LSTM             │  1.4 M │  [1, 115, 60] │ [[1, 115, 256], [[8, 1, 128], [8, 1, 128]]] │
│ 2 │ linear            │ ModuleList       │ 49.4 K │             ? │                                           ? │
│ 3 │ classifier        │ Linear           │  1.3 K │ [1, 115, 128] │                                [1, 115, 10] │
│ 4 │ activation        │ Sigmoid          │      0 │  [1, 115, 10] │                                [1, 115, 10] │
│ 5 │ validation_metric │ MetricCollection │      0 │             ? │                                           ? │
└───┴───────────────────┴──────────────────┴────────┴───────────────┴─────────────────────────────────────────────┘

Trainable params: 1.5 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 1.5 M                                                                                                
Total estimated model params size (MB): 5

Output()

MisconfigurationException: Could not find the `LightningModule` attribute for the `torchmetrics.Metric` logged. You can fix this by calling `self.log(DiarizationErrorRate, ..., metric_attribute=name)` where `name` is one of ['validation_metric.DiarizationErrorRate', 'validation_metric.DiarizationErrorRate/Confusion', 'validation_metric.DiarizationErrorRate/FalseAlarm', 'validation_metric.DiarizationErrorRate/Miss', 'validation_metric.DiarizationErrorRate/Threshold']

In [65]:
from pyannote.audio.tasks import SpeakerDiarization
seg_task = SpeakerDiarization(protocol, duration=5)

TypeError: Protocol.train() missing 1 required positional argument: 'self'

In [4]:
protocol = registry.get_protocol('MyDatabase.Protocol.MyProtocol')
for resource in protocol.train():
    print(resource["uri"])

# /path/to/train.lst
filename1
filename2


In [5]:
import json

# Your given JSON data
data_json = """{"id": 10, "created_username": " stephen.witkowski@protonmail.com, 1", "created_ago": "0\u00a0minutes", "completed_by": {"id": 1, "first_name": "", "last_name": "", "email": "stephen.witkowski@protonmail.com"}, "task": {"id": 10, "data": {"audio": "gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav"}, "meta": {}, "created_at": "2023-08-04T16:20:11.293823Z", "updated_at": "2023-08-04T16:20:11.293887Z", "is_labeled": true, "overlap": 1, "inner_id": 10, "total_annotations": 1, "cancelled_annotations": 0, "total_predictions": 0, "comment_count": 0, "unresolved_comment_count": 0, "last_comment_updated_at": null, "project": 1, "updated_by": null, "file_upload": null, "comment_authors": []}, "result": [{"original_length": 61, "value": {"start": 1.5313807531380752, "end": 4.913179916317992, "channel": 0, "labels": ["Speaker A"]}, "id": "5rUHO", "from_name": "label", "to_name": "audio", "type": "labels", "origin": "manual"}, {"original_length": 61, "value": {"start": 10.336820083682008, "end": 11.80439330543933, "channel": 0, "labels": ["Speaker B"]}, "id": "C1ziU", "from_name": "label", "to_name": "audio", "type": "labels", "origin": "manual"}, {"original_length": 61, "value": {"start": 39.87970711297071, "end": 41.85774058577406, "channel": 0, "labels": ["Speaker C"]}, "id": "hD5zJ", "from_name": "label", "to_name": "audio", "type": "labels", "origin": "manual"}, {"original_length": 61, "value": {"start": 47.98326359832636, "end": 49.70606694560669, "channel": 0, "labels": ["Speaker D"]}, "id": "t-Lct", "from_name": "label", "to_name": "audio", "type": "labels", "origin": "manual"}, {"original_length": 61, "value": {"start": 50.40794979079498, "end": 53.59832635983263, "channel": 0, "labels": ["Speaker E"]}, "id": "76TVG", "from_name": "label", "to_name": "audio", "type": "labels", "origin": "manual"}, {"original_length": 61, "value": {"start": 56.98012552301255, "end": 61, "channel": 0, "labels": ["Speaker F"]}, "id": "DzC_L", "from_name": "label", "to_name": "audio", "type": "labels", "origin": "manual"}], "was_cancelled": false, "ground_truth": false, "created_at": "2023-08-04T17:34:56.018687Z", "updated_at": "2023-08-04T17:34:56.018738Z", "draft_created_at": "2023-08-04T17:34:07.414372Z", "lead_time": 59.522, "last_action": null, "project": 1, "updated_by": 1, "parent_prediction": null, "parent_annotation": null, "last_created_by": null}""" # truncated for brevity

# Parse the JSON string
data = json.loads(data_json)

# Initialize the RTTM output string
rttm_output = ""

# Iterate through the results and format them in RTTM
for result in data["result"]:
    start_time = result["value"]["start"]
    duration = result["value"]["end"] - start_time
    speaker = result["value"]["labels"][0].split(" ")[-1] # Extracting the speaker label
    channel = result["value"]["channel"]
    
    rttm_line = f"SPEAKER {data['task']['data']['audio']} 1 {start_time:.2f} {duration:.2f} <NA> <NA> {speaker} <NA> <NA>\n"
    rttm_output += rttm_line

# Print the RTTM output
print(rttm_output)


SPEAKER gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav 1 1.53 3.38 <NA> <NA> A <NA> <NA>
SPEAKER gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav 1 10.34 1.47 <NA> <NA> B <NA> <NA>
SPEAKER gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav 1 39.88 1.98 <NA> <NA> C <NA> <NA>
SPEAKER gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav 1 47.98 1.72 <NA> <NA> D <NA> <NA>
SPEAKER gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav 1 50.41 3.19 <NA> <NA> E <NA> <NA>
SPEAKER gs://purple-label-studio-data/broadcastify/input/input_chunk_108.wav 1 56.98 4.02 <NA> <NA> F <NA> <NA>



In [19]:
from google.cloud import storage
import json
import random

def convert_to_rttm(data):
    rttm_output = ""
    for result in data["result"]:
        start_time = result["value"]["start"]
        duration = result["value"]["end"] - start_time
        speaker = result["value"]["labels"][0].split(" ")[-1]
        channel = result["value"]["channel"]
        rttm_line = f"SPEAKER {data['task']['data']['audio']} 1 {start_time:.2f} {duration:.2f} <NA> <NA> {speaker} <NA> <NA>\n"
        rttm_output += rttm_line
    return rttm_output


bucket_name = 'purple-label-studio-data'
prefix = 'broadcastify/output/'

# Initialize the GCS client
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Initialize the RTTM content for train, test, and development
rttm_outputs = {"train": "", "test": "", "development": ""}

# Collect blob contents
blob_contents = []
blobs = bucket.list_blobs(prefix=prefix)
for blob in blobs:
    if blob.name != prefix:
        print(f"Processing {blob.name}...")
        blob_content = blob.download_as_text()
        data = json.loads(blob_content)
        rttm_output = convert_to_rttm(data)
        blob_contents.append(rttm_output)

# Shuffle and split the contents
random.shuffle(blob_contents)
train_size = int(0.8 * len(blob_contents))
test_size = int(0.1 * len(blob_contents))

rttm_outputs["train"] = "\n".join(blob_contents[:train_size])
rttm_outputs["test"] = "\n".join(blob_contents[train_size:train_size + test_size])
rttm_outputs["development"] = "\n".join(blob_contents[train_size + test_size:])

# Write the RTTM content to files
for split, content in rttm_outputs.items():
    with open(f'{split}.rttm', 'w') as file:
        file.write(content)
        print(f"{split}.rttm file created successfully!")


Processing broadcastify/output/1...
Processing broadcastify/output/10...
Processing broadcastify/output/11...
Processing broadcastify/output/12...
Processing broadcastify/output/13...
Processing broadcastify/output/14...
Processing broadcastify/output/15...
Processing broadcastify/output/16...
Processing broadcastify/output/17...
Processing broadcastify/output/18...
Processing broadcastify/output/19...
Processing broadcastify/output/2...
Processing broadcastify/output/20...
Processing broadcastify/output/21...
Processing broadcastify/output/22...
Processing broadcastify/output/23...
Processing broadcastify/output/24...
Processing broadcastify/output/25...
Processing broadcastify/output/26...
Processing broadcastify/output/3...
Processing broadcastify/output/4...
Processing broadcastify/output/5...
Processing broadcastify/output/6...
Processing broadcastify/output/7...
Processing broadcastify/output/8...
Processing broadcastify/output/9...
train.rttm file created successfully!
test.rttm

In [13]:
blobs.list_blobs()

AttributeError: 'HTTPIterator' object has no attribute 'list_blobs'

In [29]:
from google.cloud import storage
import json
import os

def convert_to_rttm(data, filename):
    rttm_output = ""
    for result in data["result"]:
        start_time = result["value"]["start"]
        duration = result["value"]["end"] - start_time
        speaker = result["value"]["labels"][0].split(" ")[-1]
        channel = result["value"]["channel"]
        rttm_line = f"SPEAKER {filename} 1 {start_time:.2f} {duration:.2f} <NA> <NA> {speaker} <NA> <NA>\n"
        rttm_output += rttm_line
    return rttm_output

def download_audio_file(bucket, audio_path):
    audio_directory = 'audio'
    if not os.path.exists(audio_directory):
        os.mkdir(audio_directory)
    
    # Remove the gs:// prefix and the bucket name from the path
    blob = bucket.blob(audio_path.replace('gs://', '').split('/', 1)[1])
    local_path = os.path.join(audio_directory, os.path.basename(audio_path))
    blob.download_to_filename(local_path)
    return os.path.basename(audio_path)


bucket_name = 'purple-label-studio-data'
prefix = 'broadcastify/output/'

# Initialize the GCS client
client = storage.Client()
bucket = client.get_bucket(bucket_name)

# Initialize the RTTM content and filenames list for train, test, and development
rttm_outputs = {"train": "", "test": "", "development": ""}
filenames_lists = {"train": [], "test": [], "development": []}

# Collect blob contents
blob_contents = []
file_names = []
blobs = bucket.list_blobs(prefix=prefix)
for blob in blobs:
    if blob.name != prefix:  # Skip the directory itself
        print(f"Processing {blob.name}...")
        blob_content = blob.download_as_text()
        data = json.loads(blob_content)
        filename = os.path.basename(data['task']['data']['audio'])
        rttm_output = convert_to_rttm(data, filename)
        blob_contents.append(rttm_output)

        # Download the original audio file
        downloaded_filename = download_audio_file(bucket, data['task']['data']['audio'])
        file_names.append(downloaded_filename)

# Shuffle and split the contents
train_size = int(0.8 * len(blob_contents))
test_size = int(0.1 * len(blob_contents))

rttm_outputs["train"] = "".join(blob_contents[:train_size])
rttm_outputs["test"] = "".join(blob_contents[train_size:train_size + test_size])
rttm_outputs["development"] = "".join(blob_contents[train_size + test_size:])

filenames_lists["train"] = file_names[:train_size]
filenames_lists["test"] = file_names[train_size:train_size + test_size]
filenames_lists["development"] = file_names[train_size + test_size:]

# Create directories for rttms and lsts if they don't exist
if not os.path.exists('rttms'):
    os.mkdir('rttms')
if not os.path.exists('lsts'):
    os.mkdir('lsts')

# Write the RTTM content and filenames to files
for split, content in rttm_outputs.items():
    with open(f'rttms/{split}.rttm', 'w') as file:
        file.write(content)
        print(f"rttms/{split}.rttm file created successfully!")

    with open(f'lsts/{split}.lst', 'w') as file:
        file.write("\n".join(filenames_lists[split]))
        print(f"lsts/{split}.lst file created successfully!")

Processing broadcastify/output/1...
Processing broadcastify/output/10...
Processing broadcastify/output/11...
Processing broadcastify/output/12...
Processing broadcastify/output/13...
Processing broadcastify/output/14...
Processing broadcastify/output/15...
Processing broadcastify/output/16...
Processing broadcastify/output/17...
Processing broadcastify/output/18...
Processing broadcastify/output/19...
Processing broadcastify/output/2...
Processing broadcastify/output/20...
Processing broadcastify/output/21...
Processing broadcastify/output/22...
Processing broadcastify/output/23...
Processing broadcastify/output/24...
Processing broadcastify/output/25...
Processing broadcastify/output/26...
Processing broadcastify/output/3...
Processing broadcastify/output/4...
Processing broadcastify/output/5...
Processing broadcastify/output/6...
Processing broadcastify/output/7...
Processing broadcastify/output/8...
Processing broadcastify/output/9...
rttms/train.rttm file created successfully!
lst

In [ ]:
# instantiate loader registered with `.rttm` suffix
speaker = RTTMLoader('rttms/train.rttm')

# entries with {placeholders} serve as path templates
transcription_template = 'ctms/{uri}.ctm'

for resource in protocol.train():
    # unique resource identifier
    uri = resource['uri']

    # only select parts of `rttms/train.rttm` that are relevant to current resource,
    # convert it into a convenient data structure (here pyannote.core.Annotation), 
    # and assign it to `'speaker'` resource key 
    resource['speaker'] = speaker[uri]

    # replace placeholders in `transcription` path template
    ctm = transcription_template.format(uri=uri)

    # instantiate loader registered with `.ctm` suffix
    transcription = CTMLoader(ctm)

    # only select parts of the `ctms/{uri}.ctm` that are relevant to current resource
    # (here, most likely the whole file), convert it into a convenient data structure
    # (here spacy.tokens.Doc), and assign it to `'transcription'` resource key 
    resource['transcription'] = transcription[uri]
